# Use Model Switching to handle Azure OpenAI ReadTimeout Errors

**🚀 Do this in 1 line of code [reliableGPT](https://github.com/BerriAI/reliableGPT)**

`reliableGPT(openai.ChatCompletion.create)`

Contributions welcome 😊

## Environment Set-up

In [ ]:
!pip install openai

In [14]:
#@title #Set-up your Azure OpenAI Instance
import openai
import os
os.environ["AZURE_OPENAI_ENDPOINT"] = "YOUR_AZURE_OPENAI_ENDPOINT" #@param {type:"string"}
os.environ["AZURE_OPENAI_KEY"] = "YOUR_AZURE_OPENAI_KEY" #@param {type:"string"}
os.environ["AZURE_DEPLOYMENT_ID"] = "YOUR_AZURE_DEPLOYMENT_ID" #@param {type:"string"}
os.environ["OPENAI_API_KEY"] = "YOUR_BACKUP_OPENAI_KEY" #@param {type:"string"}
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

# Implement Model Switching

Here's how we'll do this:
- `Wrapper Function`: We'll wrap the OpenAI endpoint, to catch any errors that might occur
- `Fallback Strategy`: If an error occurs, we'll rotate through different models
- `Backup OpenAI Key`: Our fallback will be to call the raw OpenAI endpoints instead

In [9]:
import traceback
import copy
import random
import signal

# Our fallback strategy
def fallback_fn(fn, args, kwargs, fallback_strategy):
  try:
    # preserve the azure endpoint details
    azure_endpoint_details = openai.__dict__.copy()
    # get the backup openai key
    backup_openai_key = os.getenv("OPENAI_API_KEY")
    # switch to the raw openai model instead of using azure.
    openai.api_type = "openai"
    openai.api_base = "https://api.openai.com/v1"
    openai.api_version = None
    openai.api_key = backup_openai_key
    new_kwargs_except_engine = {
      k: v
      for k, v in kwargs.items()
      if k != "engine" and k!= "deployment_id"
    }
    print(f"new kwargs: {new_kwargs_except_engine}")
    for model in fallback_strategy:
      new_kwargs_except_engine['model'] = model
      completion = fn(**new_kwargs_except_engine)
      if completion != None:
        break
    # reset openai back to azure,
    openai.api_type = azure_endpoint_details["api_type"]
    openai.api_base = azure_endpoint_details["api_base"]
    openai.api_version = azure_endpoint_details["api_version"]
    openai.api_key = azure_endpoint_details["api_key"]
    return completion
  except:
    traceback.print_exc()
    return None

def timeout_handler(signum, frame):
    raise TimeoutError("Function execution timed out")

# Our Wrapper Function!!
def wrapper_fn(fn, fallback_strategy=["gpt-4", "gpt-3.5-turbo-16k", "gpt-3.5-turbo"]): # setting our default fallback strategy, in case OpenAI fails
  def wrapped_fn(*args, **kwargs):
    # Set the timeout signal handler <- to break out of requests in case they take too long to execute
    signal.signal(signal.SIGALRM, timeout_handler)

    # Set the timeout alarm for 10s
    signal.alarm(10)
    try:
      print("received request!")
      result = fn(*args, **kwargs)
    except Exception as e:
      traceback.print_exc()
      # Disable the timeout alarm
      signal.alarm(0)
      print(f"args: {args}")
      print(f"kwargs: {kwargs}")
      result = fallback_fn(fn, args, kwargs, fallback_strategy=fallback_strategy)
      if result == None:
        raise e
    return result
  return wrapped_fn

## Wrapping our OpenAI call!

In [12]:
original_fn = openai.ChatCompletion.create # save the original reference (good practice)
openai.ChatCompletion.create = wrapper_fn(openai.ChatCompletion.create)

# 🚀 Test with 100 OpenAI Calls

In [ ]:
for _ in range(5):
  # create a chat completion
  chat_completion = openai.ChatCompletion.create(deployment_id=os.getenv("AZURE_DEPLOYMENT_ID"), model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
  # print the completion
  print(chat_completion.choices[0].message.content)